In [23]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [27]:
cols = ['name', 'date', 'address', 'infoLink']
catagoryDict = {'2': 'Learning', '4': 'Performance', '11': 'Exhibition', '7': 'Mixer', '8': 'Foods', '6': 'Outdoors'}
df_name = pd.DataFrame(columns = [cols[0]])
df_date = pd.DataFrame(columns = [cols[1]])
df_address = pd.DataFrame(columns = [cols[2]])
df_info = pd.DataFrame(columns = [cols[3]])

In [24]:
def parseEventData(urlBase, catagory):
    catagoryValue = catagoryDict[catagory]
    
    res = requests.get(url + catagory)
    soup = soup = BeautifulSoup(res.text, "lxml")
    
    rawDateData = soup.select('.timezoneSuffix')
    rawEventNameData = soup.select('h2 a')
    rawInfoHref = soup.select('.btn-point')
    
    for rawDate in rawDateData:
        date = rawDate.get_text()
        df_date = df_date.append({'date': date}, ignore_index=True)

    for event in rawEventNameData:
        eventName = event.get_text()
        df_name = df_name.append({'name': eventName}, ignore_index=True)

    for a in soup.find_all('a', href=True, attrs={'class': 'btn-point btn-small pull-right'}):
        eventInfoLink = a['href']
        df_info = df_info.append({'infoLink': eventInfoLink}, ignore_index=True)
        
        infoRes = requests.get(eventInfoLink)
        infoSoup = BeautifulSoup(infoRes.text, "lxml")
        for span in infoSoup.find_all('span', attrs={'class': 'info-desc'}):
            if span.findChildren('i', attrs={'class': 'fa fa-map-marker'}):
                address = span.get_text()
                df_address = df_address.append({'address': address}, ignore_index=True)

In [ ]:
def combineData():
    df = pd.concat([df_name, df_date, df_address, df_info], axis=1)
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = df.dropna()
    df = df.to_dict()

In [25]:
catagories = ['2', '4', '11', '7', '8' ,'6']

for catagory in catagories:
    urlBased = 'https://kktix.com/events?category_id='
    parseEventData(urlBased, catagory)
    combineData()

{'name': {2: '細野晴臣 Concert Tour',
  3: '【雙人/單人預售票】ORANGE RANGE LIVE TOUR 018－019～ELEVEN PIECE～',
  4: 'ORANGE RANGE LIVE TOUR 018－019～ELEVEN PIECE～',
  5: '【OR官方FC會員限定票】ORANGE RANGE LIVE TOUR 018－019～ELEVEN PIECE～',
  6: '洛基恐怖秀新年狂歡場 Rocky Horror New Years Show',
  7: 'Deca joins ‘Go Slow’-台南_2',
  8: '2019 江美琪<我愛江美琪>演唱會 ',
  9: 'Deca joins ‘Go Slow’-台南_1'},
 'date': {2: '2019/02/23 19:30(+0800)',
  3: '2019/02/16 19:00(+0800)',
  4: '2019/02/16 19:00(+0800)',
  5: '2019/02/16 19:00(+0800)',
  6: '2019/01/12 21:00(+0800)',
  7: '2019/01/12 20:00(+0800)',
  8: '2019/01/12 19:30(+0800)',
  9: '2019/01/11 20:00(+0800)'},
 'address': {2: '西門町WESTAR  / 台北市萬華區漢中街116號8樓',
  3: 'Legacy Taipei 音樂展演空間 / 台北市八德路一段一號 華山中五A館',
  4: 'Legacy Taipei / 台北市八德路一段一號 中5A館',
  5: 'Legacy Taipei / 台北市八德路一段一號 中5A館',
  6: '京華城喜滿客戲院 / 台北市松山區八德路四段138號B1',
  7: 'TCRC / 700台南市中西區西門路二段314號',
  8: '台北國際會議中心 / 臺北市信義路五段1號',
  9: 'TCRC / 700台南市中西區西門路二段314號'},
 'infoLink': {2: 'https://bluetreepress2.kktix.cc/events/44b2e